In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pyodbc
import mysql.connector
import json
import matplotlib.pyplot as plt
import seaborn as sns
from decimal import Decimal

import requests
import datetime
from tqdm import tqdm

In [2]:
pyodbc.drivers()

['MySQL']

In [12]:
# Redshift 연결 정보
host=''
dbname=''
user=''
password=''
port=''

In [13]:
# Redshift에 연결
conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password,
    port=port
)

### 일탈에 대한 시계열 테이블 가져오기
- "data_analysis_db"."da_schema"."t_frigga_alert_target" 

In [14]:
# 커서 생성
cursor = conn.cursor()
# SQL 쿼리 실행 (예시: frigga_sample 테이블에서 모든 데이터를 조회)
cursor.execute("""                          
                    SELECT 
                        *
                    FROM 
                        "data_analysis_db"."da_schema"."t_frigga_alert_target"
                    WHERE 1=1
                    ORDER BY device_serial ASC, device_datetime_kor ASC
                ;
               """)

# 결과 가져오기
rows = cursor.fetchall()

# 컬럼명 가져오기
columns = [desc[0] for desc in cursor.description]

# DataFrame으로 변환
df_alert = pd.DataFrame(rows, columns=columns)

# 커서와 연결 종료
cursor.close()
conn.close()

### 파손에 대한 시계열 테이블 가져오기
- "data_analysis_db"."da_schema"."t_frigga_broken_target"

In [15]:
# Redshift에 연결
conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password,
    port=port
)

In [16]:
# 커서 생성
cursor = conn.cursor()
# SQL 쿼리 실행 (예시: frigga_sample 테이블에서 모든 데이터를 조회)
cursor.execute("""                          
                    SELECT 
                        *
                    FROM 
                        "data_analysis_db"."da_schema"."t_frigga_broken_target"
                    WHERE 1=1
                    ORDER BY device_serial ASC
                ;
               """)

# 결과 가져오기
rows = cursor.fetchall()

# 컬럼명 가져오기
columns = [desc[0] for desc in cursor.description]

# DataFrame으로 변환
df_broken = pd.DataFrame(rows, columns=columns)

# 커서와 연결 종료
cursor.close()
conn.close()

In [17]:
df_alert.head(4)

,device_datetime_kor,device_serial,lat,lon,battery,temperature,humidity,acc_orig,acc,accx,...,aws_lon,distance_km,ws1,ws10,ta,hm,alert_temp,alert_humi,alert_acc,item_shock
0,2024-12-02 14:47:18,VC6M6002,36.811090,127.049162,100.0,17.5,54.0,1.0,1.0,0.0,...,127.00805000,9.090292,8.10,5.30,15.30,60.60,0,0,0,0
1,2024-12-02 14:57:18,VC6M6002,36.811090,127.049162,100.0,16.8,57.0,1.1,1.1,0.0,...,127.00805000,9.090292,8.10,5.30,15.30,60.60,0,0,1,0
2,2024-12-02 15:07:18,VC6M6002,36.811090,127.049162,100.0,16.4,58.0,1.0,1.0,0.0,...,127.00805000,9.090292,6.90,3.10,15.40,59.10,0,0,0,0
3,2024-12-02 15:17:18,VC6M6002,36.811090,127.049162,100.0,15.8,59.0,1.0,1.0,0.0,...,127.00805000,9.090292,6.90,3.10,15.40,59.10,0,0,0,0


In [19]:
df_alert.to_csv('../data/df_alert_250217.csv', encoding='utf-8', index=False)

In [18]:
df_broken.head(4)

,device_serial,start_date,end_date,duration_time,duration_day,min_temp,max_temp,mean_temp,std_temp,min_humi,...,humi_end,q1_acc,q3_acc,acc_start,acc_end,item_shock,alert_temp_sum,alert_humi_sum,alert_acc_sum,broken_yn
0,VC6M6002,2024-12-02 14:47:18,2024-12-11 23:37:18,224.83,9.00,1.80,21.00,9.36,11.57,48.00,...,84.00,1.00,1.00,1.00,1.00,0,12,10,200,0
1,VC6M6002,2024-12-02 14:47:18,2024-12-11 23:37:18,224.83,9.00,1.80,21.00,9.36,11.57,48.00,...,84.00,1.00,1.00,1.00,1.00,0,12,10,200,0
2,VC6M6003,2024-12-14 09:40:13,2025-01-23 23:50:13,974.17,40.00,-0.10,19.80,6.83,14.00,37.00,...,81.50,1.00,1.01,0.98,1.02,0,24,0,148,1
3,VC6M6003,2024-12-14 09:40:13,2025-01-23 23:50:13,974.17,40.00,-0.10,19.80,6.83,14.00,37.00,...,81.50,1.00,1.01,0.98,1.02,1,24,0,148,1


In [20]:
df_broken.to_csv('../data/df_broken_250217.csv', encoding='utf-8', index=False)